In [54]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [55]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity2.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model2.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink1.csv",header=1,delim=",") |> DataFrame;

In [56]:
cell_demand = 0.001*164.98*(1.369*1e6)*2           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

# D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"];

In [197]:
# penalty = 10000 # ton/yr
penalty = 5; # ton/yr

In [58]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15];

In [59]:
ipt = []
ipt = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount = ipt

15-element Vector{Float64}:
  49670.291589732
  69512.470736544
  67696.846500888
  69382.78329114
 129687.44540400001
  72376.074329
   5239.896784
   1309.974196
  31111.887154999997
  60258.81301599999
 327493.549
   2258.5762
   1806.86096
 451715.24
 451715.24

In [60]:
D = [0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.4]*cell_demand

8-element Vector{Float64}:
 225857.62
 271029.144
 316200.66799999995
 361372.19200000004
 406543.716
 451715.24
 496886.764
 632401.3359999999

### Solve subproblem to get dual $\pi$ and $\alpha$

In [61]:
sub_nctr = nproc * ncty + size(cathode)[1]*ncty + size(cell)[1]*ncty + size(noncell)[1]*ncty + size(battery)[1]*ncty

870

In [406]:
println(["x_$(i)_$(k)" for k in 1:3 for i in 1:5])

["x_1_1", "x_2_1", "x_3_1", "x_4_1", "x_5_1", "x_1_2", "x_2_2", "x_3_2", "x_4_2", "x_5_2", "x_1_3", "x_2_3", "x_3_3", "x_4_3", "x_5_3"]


In [407]:
test = ["x_$(i)_$(k)" for k in 1:3 for i in 1:5]
reshape(test, 5, 3)

5×3 Matrix{String}:
 "x_1_1"  "x_1_2"  "x_1_3"
 "x_2_1"  "x_2_2"  "x_2_3"
 "x_3_1"  "x_3_2"  "x_3_3"
 "x_4_1"  "x_4_2"  "x_4_3"
 "x_5_1"  "x_5_2"  "x_5_3"

In [430]:
test2 = ["x_$(j)_$(10)*k[$(k)]" for k in 1:5 for j in 1:30]
reshape(test2, 30, 5)

30×5 Matrix{String}:
 "x_1_10*k[1]"   "x_1_10*k[2]"   …  "x_1_10*k[4]"   "x_1_10*k[5]"
 "x_2_10*k[1]"   "x_2_10*k[2]"      "x_2_10*k[4]"   "x_2_10*k[5]"
 "x_3_10*k[1]"   "x_3_10*k[2]"      "x_3_10*k[4]"   "x_3_10*k[5]"
 "x_4_10*k[1]"   "x_4_10*k[2]"      "x_4_10*k[4]"   "x_4_10*k[5]"
 "x_5_10*k[1]"   "x_5_10*k[2]"      "x_5_10*k[4]"   "x_5_10*k[5]"
 "x_6_10*k[1]"   "x_6_10*k[2]"   …  "x_6_10*k[4]"   "x_6_10*k[5]"
 "x_7_10*k[1]"   "x_7_10*k[2]"      "x_7_10*k[4]"   "x_7_10*k[5]"
 "x_8_10*k[1]"   "x_8_10*k[2]"      "x_8_10*k[4]"   "x_8_10*k[5]"
 "x_9_10*k[1]"   "x_9_10*k[2]"      "x_9_10*k[4]"   "x_9_10*k[5]"
 "x_10_10*k[1]"  "x_10_10*k[2]"     "x_10_10*k[4]"  "x_10_10*k[5]"
 "x_11_10*k[1]"  "x_11_10*k[2]"  …  "x_11_10*k[4]"  "x_11_10*k[5]"
 "x_12_10*k[1]"  "x_12_10*k[2]"     "x_12_10*k[4]"  "x_12_10*k[5]"
 "x_13_10*k[1]"  "x_13_10*k[2]"     "x_13_10*k[4]"  "x_13_10*k[5]"
 ⋮                               ⋱                  
 "x_19_10*k[1]"  "x_19_10*k[2]"     "x_19_10*k[4]"  "x_19_10*k[5

In [416]:
B = zeros(30,15);

In [448]:
2*ones(3)
[1,2,3] .* [3,4,5]
size(cathode)
tt = [[1,2,3,4] [2,3,4,5]]
tt[:,2] = [1,2,3,4]
tt
# transpose(tt)

4×2 Matrix{Int64}:
 1  1
 2  2
 3  3
 4  4

In [454]:
append!([1,2], [1,2,4])

5-element Vector{Int64}:
 1
 2
 1
 2
 4

In [456]:
function reformu_pi(col_idx, proc_idx, ls_cstr)
    pi_matrix = zeros(ncty, nproc)
    pi = [getdual(con) for con in ls_cstr]
    B = []
    for k in proc_idx
        append!(B, scaler[k] * ones(ncty))
    end
    pi = pi .* B
    pi = reshape(pi, ncty, size(proc_idx)[1])
    pi_matrix[:,col_idx] = sum(pi, 2)
    
    return pi_matrix
end


# cathode_pi = zeros(ncty, nproc)
# cth_pi = [getdual(con) for con in cstr_cth]
# cth_B = []
# for k in cathode
#     push!(cth_B, scaler[k] * ones(ncty))
# end
# cth_pi = cth_pi .* cth_B
# cth_pi = reshape(cth_pi, ncty, size(cathode)[1])
# cathode_pi[:,5] = sum(cth_pi, 2)

reformu_pi (generic function with 1 method)

In [457]:
function subprob(x_hat, d)
    model = Model(Gurobi.Optimizer)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, s >= 0)

    # node output flow constraint
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x_hat[i,k]) for k in 1:nproc for i in 1:ncty]

    # cathode LCA constraints (index=5)
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]

    # cell LCA constraints (index=11)
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]

    # non cell LCA constraints (index=14)
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]

    # battery LCA constraints (index=15)
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty]
    

    for j in 1:ncty
        if j == mkt_loc
            @constraint(model, cstr_alp, sum(y[i,j,mkt_proc] for i in 1:ncty) + s >= d)
        else
            @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0)
        end
    end    
    
    
    #######################
    tranD = Vector{AffExpr}(undef, ncty)
    tranS= Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
        end
        tranD[j] = arc_emi
        tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
    end
    
    
    @objective(model, Min, sum(tranD-tranS) + s*penalty)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_s = JuMP.value.(s)
    
    
    ####################### 
    op_pi = [getdual(con) for con in cstr_op]
    op_pi = reshape(op_pi, ncty, nproc)

    cth_pi = reformu_pi(5, cathode, cstr_cth)
    cell_pi = reformu_pi(11, cell, cstr_cell)
    noncell_pi = reformu_pi(14, noncell, cstr_noncell)
    battery_pi = reformu_pi(15, battery, cstr_battery)
    pi_matrix = op_pi + cth_pi + cell_pi + noncell_pi + battery_pi
    
    alp = getdual(cstr_alp)

#     ls_pi = []
#     ls_alp = []
#     for ls in ls_cstr
#         temp = []
#         for con in ls
#             push!(temp, getdual(con))
#         end
#         push!(ls_pi, temp)
#     end

    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "s_opt"=>sub_s, "pi"=>ls_pi, "alp"=>alp])
    return result
    
end


subprob (generic function with 2 methods)

In [458]:
# function subprob(x_hat, d)
#     model = Model(Gurobi.Optimizer)
#     @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
#     @variable(model, s >= 0)

#     # node output flow constraint
#     cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x_hat[i,k]) for k in 1:nproc for i in 1:ncty]

#     # cathode LCA constraints (index=5)
#     cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,5]*scaler[k]) for j in 1:ncty for k in cathode]

#     # cell LCA constraints (index=11)
#     cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,11]*scaler[k]) for j in 1:ncty for k in cell]

#     # non cell LCA constraints (index=14)
#     cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,14]*scaler[k]) for j in 1:ncty for k in noncell]

#     # battery LCA constraints (index=15)
#     cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,15]*scaler[k]) for j in 1:ncty for k in battery]

# #     ls_cstr = [cstr_op; cstr_cth; cstr_cell; cstr_noncell; cstr_battery]
#     ls_cstr = [cstr_op; cstr_cth; cstr_cell; cstr_noncell; cstr_battery]
    


#     for j in 1:ncty
#         if j == mkt_loc
#             @constraint(model, cstr_alp, sum(y[i,j,mkt_proc] for i in 1:ncty) + s >= d)
#         else
#             @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0)
#         end
#     end    
    
    
#     #######################
#     tranD = Vector{AffExpr}(undef, ncty)
#     tranS= Vector{AffExpr}(undef, ncty)
#     for j in 1:ncty
#         arc_emi = 0
#         for i in 1:ncty
#             amount = sum(y[i,j,k] for k in 1:nproc)
#             arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
#         end
#         tranD[j] = arc_emi
#         tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
#     end
    
# #     tranD = Vector{AffExpr}(undef, ncty)
# #     tranS= Vector{AffExpr}(undef, ncty)
# #     for j in 1:ncty
# #         arc_emi = 0
# #         for i in 1:ncty
# #             amount = sum(y[i,j,k] for k in 1:nproc-1)
# #             arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
# #         end
# #         tranD[j] = arc_emi
# #         tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
# #     end
# #     tranD[mkt_loc] += sum(y[i,mkt_loc, mkt_proc] for i in 1:ncty)
# #     tranS[mkt_loc] += sum(y[i,mkt_loc, mkt_proc] for i in 1:ncty) * (sink_c[j]/emission_c[j] + es_ratio)
    
#     @objective(model, Min, sum(tranD-tranS) + s*penalty)
#     JuMP.optimize!(model)
    
#     qy_hat = JuMP.objective_value(model)
#     sub_y = JuMP.value.(y)
#     sub_s = JuMP.value.(s)
    
    
#     #######################    
#     ls_pi = []
#     ls_alp = []
#     for con in ls_cstr
#         push!(ls_pi, getdual(con))
#     end

#     push!(ls_alp, getdual(cstr_alp))
    
    
#     #######################
#     result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "s_opt"=>sub_s, "pi"=>ls_pi, "alp"=>ls_alp])
#     return result
    
# end


### Solve master problem

In [103]:
M = -10e9

-1.0e10

In [111]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
        end
    end
    for k in 1:nproc
        @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    proM = x * price

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, sum(x .* G) + theta >= g)
        end
    end
    
    @objective(model, Min, sum(proD-proS)+theta)
    JuMP.optimize!(model)
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result
end

masterprob (generic function with 1 method)

### Generate cuts

In [366]:
function get_cuts(x, D)
    G = [] # gradient
    g = 0 # intersection
    qy = 0
    
    for d in D
        res = subprob(x, d)
        gradient = -1 * res["pi"]
        
        push!(G, gradient)
        g += d * res["alp"]
        qy += res["qyhat"]
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    z_hat = sum(proD - proS) + qy/size(D)[1]
    Exp_G = sum(G) / size(D)[1]
    Exp_g = sum(g) / size(D)[1]
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>z_lb])
    return cut, z_hat
end


get_cuts (generic function with 1 method)

### L Shape Method

In [113]:
function main(toler)
    i = 0
    z_ub = Inf
    x_opt = 0
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = get_cuts(x_hat, D)
        if z_hat < z_ub
            z_ub = z_hat
            x_opt = x_hat;
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    println(z_lb,z_ub, x_opt, i)
    
end

main (generic function with 1 method)

In [68]:
main(0.01)

Academic license - for non-commercial use only - expires 2023-11-27


LoadError: MethodError: no method matching +(::Matrix{AffExpr}, ::VariableRef)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  +([91m::Union{MathOptInterface.ScalarAffineFunction{T}, MathOptInterface.ScalarQuadraticFunction{T}}[39m, ::T) where T at C:\Users\bourg\.julia\packages\MathOptInterface\YDdD3\src\Utilities\functions.jl:1716
[0m  +([91m::ChainRulesCore.Tangent{P}[39m, ::P) where P at C:\Users\bourg\.julia\packages\ChainRulesCore\C73ay\src\tangent_arithmetic.jl:146
[0m  ...

----

### Debug

In [375]:
i = 0
z_ub = Inf
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 465 rows, 451 columns and 900 nonzeros
Model fingerprint: 0xd7367c88
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 3e+02]
  Bounds range     [1e+10, 1e+10]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 465 rows and 451 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0008384e+10   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -1.000838413e+10

User-callback calls 34, time in user-callback 0.00 sec


-1.0008384127042429e10

In [377]:
(z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))

true

In [378]:
new_cut, z_hat = get_cuts(x_hat, D)

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 900 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0x2b967fcc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 5e+05]
Presolve removed 900 rows and 13501 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9657527e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.965752693e+05

User-callback calls 29, time in user-callback 0.00 sec
Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, us

(Dict{String, Any}("gradient" => [-0.12781186121027854, -0.16449709458296913, -0.06253797789712207, -0.1286330854974228, -0.11155826363918264, -0.0, 0.022314648427925743, 0.6322869704056888, -0.0, -0.0  …  -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.2673616485985661], "intersection" => -1.0008384127042429e10), -8.04639076064418e6)

In [380]:
z_hat, z_ub

(-8.04639076064418e6, Inf)

In [382]:
if z_hat < z_ub
    z_ub = z_hat
    x_opt = x_hat;
end

In [386]:
push!(cuts, new_cut);

In [392]:
cuts[1]["gradient"]

870-element Vector{Float64}:
 -0.12781186121027854
 -0.16449709458296913
 -0.06253797789712207
 -0.1286330854974228
 -0.11155826363918264
 -0.0
  0.022314648427925743
  0.6322869704056888
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
  ⋮
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.2673616485985661

In [394]:
model = Model(Gurobi.Optimizer)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, theta >= M)

for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end
for k in 1:nproc
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
proM = x * price

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1);

Academic license - for non-commercial use only - expires 2023-11-27


In [400]:
cuts[1]["gradient"]

870-element Vector{Float64}:
 -0.12781186121027854
 -0.16449709458296913
 -0.06253797789712207
 -0.1286330854974228
 -0.11155826363918264
 -0.0
  0.022314648427925743
  0.6322869704056888
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
  ⋮
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.2673616485985661

In [396]:
cuts != []

true

In [398]:
if cuts != []
    for cut in cuts
        G = cut["gradient"]
        g = cut["intersection"]
        println(G)
#         @constraint(model, sum(x .* G) + theta >= g)
    end
end

[-0.12781186121027854, -0.16449709458296913, -0.06253797789712207, -0.1286330854974228, -0.11155826363918264, -0.0, 0.022314648427925743, 0.6322869704056888, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.1135170989960177, 1.461923867459026, -0.0, 1.1897393305285533, 0.08761891679301259, -0.0, 0.868654925031171, -0.0, -0.0, -0.0, 1.0123247094337913, -0.0, 2.6613651079346035, -0.0, 0.08988994647713167, -0.0, -0.0, -0.1286330854974228, -0.0, -0.0, 0.24001645611533595, -0.1314602700924703, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.6981766269608674, -0.0, 0.4259920900303946, -0.0, -0.0, 0.18690434907959563, -0.0, -0.0, -0.0, 0.2485774689356326, 0.030756483002560397, 1.8976178674364448, -0.0, 3.312156049481385, 2.762316726407633, -0.0, 3.0936330175068316, 2.8009854267574643, 2.23934848244625, 3.46228255911959, 3.5591007913962907, 3.0104056146257494, 1.0935486489314261, 2.753631555740272, 0.7450768336284482, 0.9294584906210154, 1.2113022959137754, 0.800400687757687, 1.329699

, 2.5253674573908405, 0.7002876748240238, 0.9920341527023309, 1.4196255992076028, 1.219861808879065, 0.2994890604272764, 0.2994890604272764, 1.218777640063858, -0.0, 0.13722581243141574, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.08005756100656714, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.09844265330528067, -0.0, -0.0, -0.0, -0.0, -0.0, 0.025576663025165203, -0.0, -0.0, -0.0, -0.0, 1.2805211892380282, 0.38673633063513974, 0.18697254030660201, 2.9879579670640664, 2.9879579670640664, -0.0, 0.6198793501156351, 0.5992297200781769, 0.3994659297496392, 0.8522695063621838, 0.8522695063621838, 0.36874426333437244, -0.0, 0.18915265768554604, -0.0, -0.0, -0.0, -0.0, -0.0, 0.01258342124653039, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0

[-0.12781186121027854, -0.16449709458296913, -0.06253797789712207, -0.1286330854974228, -0.11155826363918264, -0.0, 0.022314648427925743, 0.6322869704056888, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.1135170989960177, 1.461923867459026, -0.0, 1.1897393305285533, 0.08761891679301259, -0.0, 0.868654925031171, -0.0, -0.0, -0.0, 1.0123247094337913, -0.0, 2.6613651079346035, -0.0, 0.08988994647713167, -0.0, -0.0, -0.1286330854974228, -0.0, -0.0, 0.24001645611533595, -0.1314602700924703, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.6981766269608674, -0.0, 0.4259920900303946, -0.0, -0.0, 0.18690434907959563, -0.0, -0.0, -0.0, 0.2485774689356326, 0.030756483002560397, 1.8976178674364448, -0.0, 3.312156049481385, 2.762316726407633, -0.0, 3.0936330175068316, 2.8009854267574643, 2.23934848244625, 3.46228255911959, 3.5591007913962907, 3.0104056146257494, 1.0935486489314261, 2.753631555740272, 0.7450768336284482, 0.9294584906210154, 1.2113022959137754, 0.800400687757687, 1.329699

, 0.31176960610069515, 0.05925323632385182, 0.252054778723741, 0.052290988395203186, 0.07690588138557342, 0.07690588138557342, 0.05247656862268668, 0.6976874650196861, 0.31483284895792657, -0.0, 1.8230831245458836, 1.8230831245458836, 0.11767659446696567, 1.4363512612893852, 2.2637597828958325, 2.0151292156331126, 0.19460203067680698, 0.19460203067680698, 2.066603528404872, 6.152135612839154, 9.493107025393028, 9.011841390566396, -0.0, -0.0, 9.295950770902067, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.7942942069794093, 1.3851972928580136, 1.1440615522302977, 0.24392175429945184, 0.24392175429945184, 1.188041038367053, 1.4563824106955574, 0.983887683791484, 0.7841238934629462, 2.5253674573908405, 2.5253674573908405, 0.7002876748240238, 0.9920341527023309, 1.4196255992076028, 1.219861808879065, 0.2994890604272764, 0.2994890604272764, 1.218777640063858, -0.0, 0.13722581243141574, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0800575610

In [387]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
        end
    end
    for k in 1:nproc
        @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    proM = x * price

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, sum(x .* G) + theta >= g)
        end
    end
    
    @objective(model, Min, sum(proD-proS)+theta)
    JuMP.optimize!(model)
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result
end

Academic license - for non-commercial use only - expires 2023-11-27


LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 30 and 870

In [ ]:
while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
    new_cut, z_hat = get_cuts(x_hat, D)
    if z_hat < z_ub
        z_ub = z_hat
        x_opt = x_hat
    end

    push!(cuts, new_cut)

    res1 = masterprob(cuts)
    z_lb = res1["z_lb"]
    x_hat = res1["x_hat"]

    i += 1
end

---

In [331]:
toler = 0.01

0.01

In [332]:
i = 0
z_ub = Inf
x_opt = 0

cuts = []
res0 = masterprob(cuts)

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 465 rows, 451 columns and 900 nonzeros
Model fingerprint: 0xd7367c88
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 3e+02]
  Bounds range     [1e+10, 1e+10]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 465 rows and 451 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0008384e+10   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -1.000838413e+10

User-callback calls 34, time in user-callback 0.00 sec


Dict{String, Any} with 2 entries:
  "x_hat" => [6200.0 0.0 … 0.0 0.0; 1770.29 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.…
  "z_lb"  => -1.00084e10

In [333]:
x_hat = res0["x_hat"]
z_lb = res0["z_lb"];

In [346]:
d = D[1]

225857.62

In [335]:
res = subprob(x_hat, d);

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 900 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0x2b967fcc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 5e+05]
Presolve removed 900 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9657527e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.965752693e+05

User-callback calls 29, time in user-callback 0.00 sec


In [348]:
G = [] # gradient
g = 0 # intersection
qy = 0

for d in D
    res = subprob(x_hat, d)
    gradient = -1 * res["pi"]

    push!(G, gradient)
    g += d * res["alp"][1]
    qy += res["qyhat"]
end
    

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 900 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0x2b967fcc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 5e+05]
Presolve removed 900 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9657527e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.965752693e+05

User-callback calls 29, time in user-callback 0.00 sec
Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, us

In [349]:
proD = (x_hat .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end

In [361]:
proS = (x_hat .*pro_sink)*ones(nproc,1)
z_hat = sum(proD - proS) + qy/size(D)[1]
Exp_G = sum(G) / size(D)[1]
Exp_g = sum(g) / size(D)[1]

0.0

In [368]:
res = get_cuts(x_hat, D)

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 900 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0x2b967fcc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 5e+05]
Presolve removed 900 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9657527e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.965752693e+05

User-callback calls 29, time in user-callback 0.00 sec
Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, us

(Dict{String, Any}("gradient" => [-0.12781186121027854, -0.16449709458296913, -0.06253797789712207, -0.1286330854974228, -0.11155826363918264, -0.0, 0.022314648427925743, 0.6322869704056888, -0.0, -0.0  …  -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.2673616485985661], "intersection" => -1.0008384127042429e10), -8.04639076064418e6)

In [369]:
res

(Dict{String, Any}("gradient" => [-0.12781186121027854, -0.16449709458296913, -0.06253797789712207, -0.1286330854974228, -0.11155826363918264, -0.0, 0.022314648427925743, 0.6322869704056888, -0.0, -0.0  …  -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.2673616485985661], "intersection" => -1.0008384127042429e10), -8.04639076064418e6)

---

### Draft

In [292]:
# new_cut, z_hat = get_cuts(x_hat, D)

In [126]:
G = []
g = 0 
qy = 0

0

In [178]:
d = D[1]

225857.62

In [227]:
model = Model(Gurobi.Optimizer)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, s >= 0)

# node output flow constraint
cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x_hat[i,k]) for k in 1:nproc for i in 1:ncty]

# cathode LCA constraints (index=5)
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,5]*scaler[k]) for j in 1:ncty for k in cathode]

# cell LCA constraints (index=11)
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,11]*scaler[k]) for j in 1:ncty for k in cell]

# non cell LCA constraints (index=14)
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,14]*scaler[k]) for j in 1:ncty for k in noncell]

# battery LCA constraints (index=15)
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,15]*scaler[k]) for j in 1:ncty for k in battery]

cstr1 = [cstr_op; cstr_cth; cstr_cell; cstr_noncell; cstr_battery]
ncstr1 = size(cstr1)[1]
for i in 1:ncstr1
    set_name(cstr1[i], "cstr_pi$(i)")
end

for j in 1:ncty
    if j == mkt_loc
        @constraint(model, cstr_alp, sum(y[i,j,15] for i in 1:ncty) + s >= d)
    else
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0)
    end
end


# for i in 1:ncty
#     for j in [e for e in 1:ncty if e != mkt_loc]
#         @constraint(model, y[i,j,mkt_proc] == 0)
#     end
# end

# @constraint(model, cstr_alp, sum(y[i,mkt_loc,15] for i in 1:ncty) + s >= d)

Academic license - for non-commercial use only - expires 2023-11-27


In [228]:
tranD = Vector{AffExpr}(undef, ncty)
tranS= Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
    tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
end

# tranD[mkt_loc] += sum(y[i,mkt_loc, mkt_proc] for i in 1:ncty)
# tranS[mkt_loc] += sum(y[i,mkt_loc, mkt_proc] for i in 1:ncty) * (sink_c[mkt_loc]/emission_c[mkt_loc] + es_ratio);

In [229]:
@objective(model, Min, sum(tranD-tranS) + s*penalty)
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 900 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0x2b967fcc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 5e+05]
Presolve removed 900 rows and 13501 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9657527e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.965752693e+05

User-callback calls 29, time in user-callback 0.00 sec


In [230]:
qy_hat = JuMP.objective_value(model)
sub_y = JuMP.value.(y)
sub_s = JuMP.value.(s);

In [272]:
model = Model(Gurobi.Optimizer)
@variable(model, y[1:2, 1:2, 1:3] >= 0)
@variable(model, s >= 0)
cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:2) == x_hat[i,k]) for k in 1:2 for i in 1:2]

# node output flow constraint
# @constraint(model, cstr_pi1,y[1,1,1]+y[1,2,1] == 6200) 
# @constraint(model, cstr_pi2,y[2,1,1]+y[2,2,1] == 1770.29) 
# @constraint(model, cstr_pi3,y[1,1,2]+y[1,2,2] == 0) 

Academic license - for non-commercial use only - expires 2023-11-27


4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 y[1,1,1] + y[1,2,1] == 6200.0
 y[2,1,1] + y[2,2,1] == 1770.291589731998
 y[1,1,2] + y[1,2,2] == 0.0
 y[2,1,2] + y[2,2,2] == 0.0

In [274]:
con_test = @constraint(model, y[2,1,1] >= 12)

y[2,1,1] >= 12.0

In [275]:
set_name(cstr_op[1], "cstr_pi1")
set_name(cstr_op[2], "cstr_pi2")
set_name(cstr_op[3], "cstr_pi3")

In [276]:
@objective(model, Max, sum(y[1,1,1]+y[2,1,1]))
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 5 rows, 13 columns and 9 nonzeros
Model fingerprint: 0xf3bfd4b7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 6e+03]
Presolve removed 5 rows and 13 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.9702916e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.970291590e+03

User-callback calls 25, time in user-callback 0.00 sec


In [283]:
test = []
for con in cstr_op
    push!(test, getdual(con))
end

In [284]:
test

4-element Vector{Any}:
 -1.0
 -1.0
  0.0
  0.0

In [267]:
typeof(cstr_pi1)

ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}

In [257]:
for i in 3:4
    set_name(cstr_op[i], "cstr_pi$(i)")
end

In [259]:
cstr_op[3]

cstr_pi3 : y[1,1,2] + y[1,2,2] == 0.0

In [255]:
@objective(model, Max, sum(y[1,1,1]+y[2,1,1]))
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4 rows, 13 columns and 8 nonzeros
Model fingerprint: 0x9658fdf8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+03]
Presolve removed 4 rows and 13 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.9702916e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  7.970291590e+03

User-callback calls 25, time in user-callback 0.00 sec


In [11]:
#variables 
ncty = 5
nproc = 4
@variable(model, x[1:ncty, 1:nproc])             
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0); 

In [12]:
sum(y[i,3,4] for i in 1:ncty)

y[1,3,4] + y[2,3,4] + y[3,3,4] + y[4,3,4] + y[5,3,4]

In [25]:
j = 3
k = 4
# sum(y[i,3,4] for i in 1:ncty) == x[3,15] * scaler[k]

4

In [28]:
sum(y[i,3,4] for i in 1:ncty) 

y[1,3,4] + y[2,3,4] + y[3,3,4] + y[4,3,4] + y[5,3,4]

In [29]:
sum(y[3,j,4] for j in 1:ncty)

y[3,1,4] + y[3,2,4] + y[3,3,4] + y[3,4,4] + y[3,5,4]

In [27]:
x[i,k]

x[3,4]

In [26]:
sum(x[i,4] for i in 1:ncty)

x[1,4] + x[2,4] + x[3,4] + x[4,4] + x[5,4]

In [19]:
tranD = Vector{AffExpr}(undef, ncty)
tranS= Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:3)
        arc_emi += amount
    end
    println(arc_emi)
    tranD[j] = arc_emi
    tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
end

y[1,1,1] + y[1,1,2] + y[1,1,3] + y[2,1,1] + y[2,1,2] + y[2,1,3] + y[3,1,1] + y[3,1,2] + y[3,1,3] + y[4,1,1] + y[4,1,2] + y[4,1,3] + y[5,1,1] + y[5,1,2] + y[5,1,3]
y[1,2,1] + y[1,2,2] + y[1,2,3] + y[2,2,1] + y[2,2,2] + y[2,2,3] + y[3,2,1] + y[3,2,2] + y[3,2,3] + y[4,2,1] + y[4,2,2] + y[4,2,3] + y[5,2,1] + y[5,2,2] + y[5,2,3]
y[1,3,1] + y[1,3,2] + y[1,3,3] + y[2,3,1] + y[2,3,2] + y[2,3,3] + y[3,3,1] + y[3,3,2] + y[3,3,3] + y[4,3,1] + y[4,3,2] + y[4,3,3] + y[5,3,1] + y[5,3,2] + y[5,3,3]
y[1,4,1] + y[1,4,2] + y[1,4,3] + y[2,4,1] + y[2,4,2] + y[2,4,3] + y[3,4,1] + y[3,4,2] + y[3,4,3] + y[4,4,1] + y[4,4,2] + y[4,4,3] + y[5,4,1] + y[5,4,2] + y[5,4,3]
y[1,5,1] + y[1,5,2] + y[1,5,3] + y[2,5,1] + y[2,5,2] + y[2,5,3] + y[3,5,1] + y[3,5,2] + y[3,5,3] + y[4,5,1] + y[4,5,2] + y[4,5,3] + y[5,5,1] + y[5,5,2] + y[5,5,3]


In [20]:
tranD

5-element Vector{AffExpr}:
 y[1,1,1] + y[1,1,2] + y[1,1,3] + y[2,1,1] + y[2,1,2] + y[2,1,3] + y[3,1,1] + y[3,1,2] + y[3,1,3] + y[4,1,1] + y[4,1,2] + y[4,1,3] + y[5,1,1] + y[5,1,2] + y[5,1,3]
 y[1,2,1] + y[1,2,2] + y[1,2,3] + y[2,2,1] + y[2,2,2] + y[2,2,3] + y[3,2,1] + y[3,2,2] + y[3,2,3] + y[4,2,1] + y[4,2,2] + y[4,2,3] + y[5,2,1] + y[5,2,2] + y[5,2,3]
 y[1,3,1] + y[1,3,2] + y[1,3,3] + y[2,3,1] + y[2,3,2] + y[2,3,3] + y[3,3,1] + y[3,3,2] + y[3,3,3] + y[4,3,1] + y[4,3,2] + y[4,3,3] + y[5,3,1] + y[5,3,2] + y[5,3,3]
 y[1,4,1] + y[1,4,2] + y[1,4,3] + y[2,4,1] + y[2,4,2] + y[2,4,3] + y[3,4,1] + y[3,4,2] + y[3,4,3] + y[4,4,1] + y[4,4,2] + y[4,4,3] + y[5,4,1] + y[5,4,2] + y[5,4,3]
 y[1,5,1] + y[1,5,2] + y[1,5,3] + y[2,5,1] + y[2,5,2] + y[2,5,3] + y[3,5,1] + y[3,5,2] + y[3,5,3] + y[4,5,1] + y[4,5,2] + y[4,5,3] + y[5,5,1] + y[5,5,2] + y[5,5,3]

In [21]:
sum(y[i,3,4] for i in 1:ncty)

y[1,3,4] + y[2,3,4] + y[3,3,4] + y[4,3,4] + y[5,3,4]

In [22]:
tranD[3] += sum(y[i,3,4] for i in 1:ncty)
tranD

5-element Vector{AffExpr}:
 y[1,1,1] + y[1,1,2] + y[1,1,3] + y[2,1,1] + y[2,1,2] + y[2,1,3] + y[3,1,1] + y[3,1,2] + y[3,1,3] + y[4,1,1] + y[4,1,2] + y[4,1,3] + y[5,1,1] + y[5,1,2] + y[5,1,3]
 y[1,2,1] + y[1,2,2] + y[1,2,3] + y[2,2,1] + y[2,2,2] + y[2,2,3] + y[3,2,1] + y[3,2,2] + y[3,2,3] + y[4,2,1] + y[4,2,2] + y[4,2,3] + y[5,2,1] + y[5,2,2] + y[5,2,3]
 y[1,3,1] + y[1,3,2] + y[1,3,3] + y[2,3,1] + y[2,3,2] + y[2,3,3] + y[3,3,1] + y[3,3,2] + y[3,3,3] + y[4,3,1] + y[4,3,2] + y[4,3,3] + y[5,3,1] + y[5,3,2] + y[5,3,3] + y[1,3,4] + y[2,3,4] + y[3,3,4] + y[4,3,4] + y[5,3,4]
 y[1,4,1] + y[1,4,2] + y[1,4,3] + y[2,4,1] + y[2,4,2] + y[2,4,3] + y[3,4,1] + y[3,4,2] + y[3,4,3] + y[4,4,1] + y[4,4,2] + y[4,4,3] + y[5,4,1] + y[5,4,2] + y[5,4,3]
 y[1,5,1] + y[1,5,2] + y[1,5,3] + y[2,5,1] + y[2,5,2] + y[2,5,3] + y[3,5,1] + y[3,5,2] + y[3,5,3] + y[4,5,1] + y[4,5,2] + y[4,5,3] + y[5,5,1] + y[5,5,2] + y[5,5,3]

In [8]:
for k in 1:nproc-1
    println(k)
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [32]:
tranD = Vector{AffExpr}(undef, ncty)
tranS= Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc-1)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
    tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
end

In [37]:
sum(tranD - tranS)

-0.2749938833822254 y[2,1,1] - 0.2749938833822254 y[2,1,2] - 0.2987886030573461 y[3,1,1] - 0.2987886030573461 y[3,1,2] - 0.05325282213384844 y[4,1,1] - 0.05325282213384844 y[4,1,2] - 0.06329501141503446 y[5,1,1] - 0.06329501141503446 y[5,1,2] - 0.06271886404917937 y[1,2,1] - 0.06271886404917937 y[1,2,2] - 0.052290988395203186 y[3,2,1] - 0.052290988395203186 y[3,2,2] - 0.07417425640623865 y[4,2,1] - 0.07417425640623865 y[4,2,2] - 0.07690588138557342 y[5,2,1] - 0.07690588138557342 y[5,2,2] - 2.110582919322233 y[1,3,1] - 2.110582919322233 y[1,3,2] - 1.6195340836983962 y[2,3,1] - 1.6195340836983962 y[2,3,2] - 2.013316311838815 y[4,3,1] - 2.013316311838815 y[4,3,2] - 1.8230831245458838 y[5,3,1] - 1.8230831245458838 y[5,3,2] - 0.37650600460474426 y[1,4,1] - 0.37650600460474426 y[1,4,2] - 2.299361895348044 y[2,4,1] - 2.299361895348044 y[2,4,2] - 2.015129215633112 y[3,4,1] - 2.015129215633112 y[3,4,2] - 0.19460203067680695 y[5,4,1] - 0.19460203067680695 y[5,4,2] - 2.210115357178231 y[1,5,1] - 

In [39]:
tranD[1]

1.276832997975 y[2,1,1] + 1.276832997975 y[2,1,2] + 1.38731503083 y[3,1,1] + 1.38731503083 y[3,1,2] + 0.247259901564 y[4,1,1] + 0.247259901564 y[4,1,2] + 0.29388711555299996 y[5,1,1] + 0.29388711555299996 y[5,1,2]

In [40]:
sum(y[1,1,k] for k in 1:nproc)

y[1,1,1] + y[1,1,2]

In [13]:
# ["c_$(i)_$(j)" for i in 1:3 for j in 5:7]

#     for i in 1:nproc*ncty
#         set_name(cstr_cth[i], "cathode[$(i)]")
#     end

#     for k in cell
#         for j in 1:ncty
#             @constraint(model, "lca_$(j)_$(k)", sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
#         end
#     end

#     for k in noncell
#         for j in 1:ncty
#             @constraint(model, "lca_$(j)_$(k)", sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
#         end
#     end

#     for k in battery
#         for j in 1:ncty
#             @constraint(model, "lca_$(j)_$(k)", sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
#         end
#     end

# for k in vcat(cathode, cell, noncell, battery)
#     for j in 1:ncty
#         push!(pi, getdual("cstr_demand"))
#     end
# end

In [23]:
# c1_cstr = [@constraint(model, 
#         sum(y[i,j,k] for j in 1:ncty) == x[i,k]) 
#     for k in 1:nproc for i in 1:ncty];


# for i in 1:nproc*ncty
#         set_name(c1_cstr[i], "constraint[$(i)]")
# end

In [ ]:
#     @constraint(model, cstr_alp, sum(y[i,j,k] for i in 1:ncty) + s == d)
#     ncstr1 = size(cstr1)[1]
#     for i in 1:ncstr1
#         set_name(cstr1[i], "cstr_pi[$(i)]")
#     end